In [86]:
# ! git clone https://github.com/Angeluz-07/audio-processing-files.git
# ! ls

In [ ]:
# !wget "https://www.dropbox.com/s/4jw31k5mlzcmgis/genres.tar.gz?"
# !tar xvf "/content/genres.tar.gz?" -C "/content/"

In [218]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [153]:
# from glob import glob

# #directories of normal audios
# nowvar = "rock"
# norm_data_dir = '/content/genres/'+nowvar+'/'
# norm_audio_files = glob(norm_data_dir + '*.au')

In [154]:
# import librosa
# from librosa import feature
# import numpy as np

# fn_list_i = [
#              feature.chroma_stft, feature.spectral_centroid,
#              feature.spectral_bandwidth, feature.spectral_rolloff
#             ]

# fn_list_ii = [librosa.feature.rms, feature.zero_crossing_rate]

# def get_feature_vector(y,sr): 
#    feat_vect_i = [ np.mean(funct(y,sr)) for funct in fn_list_i]
#    feat_vect_ii = [np.mean(funct(y)) for funct in fn_list_ii] 
#    feature_vector = feat_vect_i + feat_vect_ii 
#    return feature_vector

In [155]:
# norm_audios_feat = []
# for file in norm_audio_files:
#    y , sr = librosa.load(file, sr=None)
#    feature_vector = get_feature_vector(y, sr)
#    norm_audios_feat.append(feature_vector) 

In [156]:
# import csv
# norm_output = nowvar+'.csv'
# header =[
#  'chroma_stft',
#  'spectral_centroid',
#  'spectral_bandwidth',
#  'spectral_rolloff',
#  'rmse',
#  'zero_crossing_rate'
# ]
# with open(norm_output,'+w') as f:
#  csv_writer = csv.writer(f, delimiter = ',')
#  csv_writer.writerow(header)
#  csv_writer.writerows(norm_audios_feat)

In [221]:
blues = pd.read_csv("/content/blues.csv")
classical = pd.read_csv("/content/classical.csv")
country = pd.read_csv("/content/country.csv")
disco = pd.read_csv("/content/disco.csv")
hiphop = pd.read_csv("/content/hiphop.csv")
jazz = pd.read_csv("/content/jazz.csv")
metal = pd.read_csv("/content/metal.csv")
pop = pd.read_csv("/content/pop.csv")
reggae = pd.read_csv("/content/reggae.csv")
rock = pd.read_csv("/content/rock.csv")

In [188]:
final_df = pd.DataFrame(columns = rock.columns)

In [192]:
genres = [blues, 
          classical, 
          country, 
          disco, 
          hiphop, 
          jazz,
          metal, 
          pop,
          reggae,
          rock]

final_df = pd.concat(genres)

In [205]:
final_df.sample(20)

In [213]:
convertor = LabelEncoder()
class_list = final_df.iloc[:,-1]
y = convertor.fit_transform(class_list)

In [219]:
fit = StandardScaler()
X = fit.fit_transform(np.array(final_df.iloc[:,1:-1], dtype = float))

In [220]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33)